# NMR vs EPR vs muSR
## Zero Field Splitting (with only anisotropic dipolar coupling)

### NMR zero field splitting
expect 2 peaks with a splitting of 3D

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr
import netCDF4 as nc
from utils import stick_spectrum


In [ ]:
def add_doubleheaded_arrow(fig, x1, x2, y, label, label_offset=0.01):
    # Add line (arrow body)
    fig.add_shape(
        type="line",
        x0=x1, y0=y,
        x1=x2, y1=y,
        line=dict(color="black", width=1.5)
    )

    # Add left arrowhead (pointing right)
    fig.add_annotation(
        x=x1, y=y,
        ax=x1 + 20, ay=y,
        showarrow=True,
        arrowhead=3,
        arrowsize=1.2,
        arrowwidth=1.5,
        arrowcolor="black"
    )

    # Add right arrowhead (pointing left)
    fig.add_annotation(
        x=x2, y=y,
        ax=x2 - 20, ay=y,
        showarrow=True,
        arrowhead=3,
        arrowsize=1.2,
        arrowwidth=1.5,
        arrowcolor="black"
    )

    # Add centered text
    fig.add_annotation(
        x=(x1 + x2) / 2,
        y=y + label_offset,
        text=label,
        showarrow=False,
        font=dict(size=20)
    )


In [ ]:
xarr = xr.load_dataset('Data/test2.nc')
# print(ds)
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

def lorentzian(x, x0, amp, gamma):
    """
    Lorentzian centered at x0 with width gamma and amplitude amp.
    gamma: FWHM
    """
    return amp * (0.5*gamma)**2 / ((x - x0)**2 + (0.5*gamma)**2)

# Define the frequency axis for plotting
freq_min, freq_max = -2, 2
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)

# Width of each Lorentzian (FWHM), can be scalar or array
gamma = 0.01  # GHz

# Initialize spectrum
spectrum = np.zeros_like(freq_axis)

# Sum Lorentzians
for f, a in zip(freq, amps):
    if np.isfinite(f) and np.isfinite(a):
        spectrum += lorentzian(freq_axis, f, a, gamma)

# Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2)))
fig.update_layout(
    xaxis_title='𝜈',
    yaxis_title='Amplitude',
    xaxis=dict(
        showticklabels=False,   # remove x ticks
        showgrid=False,
        ticks='',               # remove tick marks
        title_standoff=8,        # decrease distance between x label and axis
        minor=dict(ticks='', showgrid=False)  # remove minor ticks and minor grid
    ),
    margin=dict(l=90, r=20, t=20, b=30),
    height=400, width=600,
)
# Example: double-headed arrow between two peaks
f1, f2 = freq[0], freq[2]
y = max(spectrum) * 0.8  # base height of the arrow
label_offset = -0.006  # label above arrow
# TODO: find out why arrowsize does not change arrow size and arrowhead only displays when arrow_size is large enough (wtf do ax and ay do?)
arrow_size = 20
label = r"$\frac{1}{2} D$"

add_doubleheaded_arrow(fig, f1, f2, y, label, label_offset=label_offset)

fig.show(renderer="browser")
